In [1]:
#!pip install -U openai

# Resources

Based om Module 2 of the Deep Learning course below (LangGraph)

* [A simple Python implementation of the ReAct pattern for LLMs](https://arc.net/l/quote/duflzttq)
  * Simon Willison Blog Article
* [Deep Learning AI Course, AI Agents with LangGraph](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/1/introduction)

### Assumptions

* Pythons setup (I created a 3.11 venv)
* OpenAI Key (Granite struggled but perhaps not a fair comparions ollama/grantite q4 8b v `gpt-4o`

In [2]:
import openai
import re
import httpx
import os
import rich

from openai import OpenAI

In [3]:
# Boilerplate for swapping in Granite via ollama
# model = "granite3-dense:8b"
# client = OpenAI(
#     base_url='http://localhost:11434/v1',
#     api_key='ollama',
# ) 

model = "gpt-4o"
client = OpenAI() 

In [4]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4o",
            temperature=0,
            messages=self.messages)
        return completion.choices[0].message.content

# ReAct Agent Prompt

In [5]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run through one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_provision_status:
e.g. get_provision_status: guid
returns the status of a cloud deployment such as a virtual machine when gived a guid (globally unique identifier)

log_error:
e.g log_error: status
When a guid has a provision_status ERROR call this with the return value of get_provision_status

log_status:
e.g log_status: status
When a guid does not have an ERROR status call this with the return value of get_provision_status

Example session:

Question: What is the staus of cloud deployment with guid <guid>
Thought: I should look up the status with get_provision_status 
Action: get_provision_status: guid 
PAUSE:

You will be called again with this:

Observation: Guid status "SUCCESS: Completed"

You then call any necessary logging tools before outputing the status:

Answer: Guid status "SUCCESS: Completed"
""".strip()

In [6]:
import random

'''
First of the *fake* functions to test if the LLM/Prompt will ReAct correctly
taking different paths on different results
'''

def get_provision_status(guid):

    #// call to MCP Server AAP2 Controller
    # // foo = bar()
    
    status_messages = [
        "INFO: Initializing",
        "INFO: In progress",
        "ERROR: Failed",
        "ERROR: API Timeout",
        "ERROR: Rate Limited",
        "WARNING: Minor errors",
        "SUCCESS: Completed"
    ]
        # "INFO: Finalizing",
    # return random.choice(f"{guid} status: {status_messages}")
    status = random.choice(status_messages)
    return f"{guid} status: {status}"
    

In [7]:
def log_error(status):
    print(f"{status} Logged stateus to Slack.")
    print(f"{status} Opened Jira Ticket with Status.")
    return 0

def log_status(status):
    print(f"{status} Logged status to Slack.")
    return 0

In [8]:
known_actions = {
   "log_status": log_status,
   "log_error": log_error,
   "get_provision_status": get_provision_status,
}

In [9]:
abot = Agent(prompt)

In [10]:
for i, message in  enumerate(abot.messages):
    if message["role"] != "assistant":
        print(f"Step {i}: App -> LLM:\n")
    else:
        print(f"Step {i}: App <- LLM:\n")
    print(f"Role: {message['role']}\nContent:\n\n{message['content']}\n\n\n")

Step 0: App -> LLM:

Role: system
Content:

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run through one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_provision_status:
e.g. get_provision_status: guid
returns the status of a cloud deployment such as a virtual machine when gived a guid (globally unique identifier)

log_error:
e.g log_error: status
When a guid has a provision_status ERROR call this with the return value of get_provision_status

log_status:
e.g log_status: status
When a guid does not have an ERROR status call this with the return value of get_provision_status

Example session:

Question: What is the staus of cloud deployment with guid <guid>
Thought: I should look up the status with get_provision_status 
Action: get_provision_sta

In [11]:
abot = Agent(prompt)
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run through one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\nget_provision_status:\ne.g. get_provision_status: guid\nreturns the status of a cloud deployment such as a virtual machine when gived a guid (globally unique identifier)\n\nlog_error:\ne.g log_error: status\nWhen a guid has a provision_status ERROR call this with the return value of get_provision_status\n\nlog_status:\ne.g log_status: status\nWhen a guid does not have an ERROR status call this with the return value of get_provision_status\n\nExample session:\n\nQuestion: What is the staus of cloud deployment with guid <guid>\nThought: I should look up the status with get_provision_status \nAction: get

In [12]:
for i, message in  enumerate(abot.messages):
    if message["role"] != "assistant":
        print(f"Step {i}: App -> LLM:\n")
    else:
        print(f"Step {i}: App <- LLM:\n")
    print(f"Role: {message['role']}\nContent:\n\n{message['content']}\n\n\n")

Step 0: App -> LLM:

Role: system
Content:

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run through one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_provision_status:
e.g. get_provision_status: guid
returns the status of a cloud deployment such as a virtual machine when gived a guid (globally unique identifier)

log_error:
e.g log_error: status
When a guid has a provision_status ERROR call this with the return value of get_provision_status

log_status:
e.g log_status: status
When a guid does not have an ERROR status call this with the return value of get_provision_status

Example session:

Question: What is the staus of cloud deployment with guid <guid>
Thought: I should look up the status with get_provision_status 
Action: get_provision_sta

# So far a bit to much "Human in the Loop"

Let's automate all this

In [13]:
action_re = re.compile("^Action: (\w+): (.*)$")

In [14]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    print("Step 0")
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            print(f"\nStep {i}")
            # print(f"Actions:\n\n{actions}")
            action, action_inputs = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: -- running {action} {action_inputs}")
            observation = known_actions[action](action_inputs)
            print(f"Observation: {observation}")
            next_prompt = f"Observation: {observation}"
        else:
            return
        

In [15]:
question = """
I have a deployments running with guid: 1adr4 what is its status
"""

query(question)

Step 0
Thought: I should look up the status of the deployment with the provided guid using get_provision_status.
Action: get_provision_status: 1adr4 
PAUSE.

Step 1
Observation: 1adr4  status: SUCCESS: Completed
Action: log_status: SUCCESS: Completed
PAUSE.

Step 2
SUCCESS: Completed Logged status to Slack.
Observation: 0
Answer: Guid status "SUCCESS: Completed"


In [16]:

question = """
I have deployments running with guids: 1adr4, aabf5, 45663, and 45ghb
First get each provision status and log to any services that need to know
Once finished with all deployments output their guids, status, and logging services:

* In a simple table
* As JSON 
"""

query(question, max_turns=10)

Step 0
Thought: I will start by checking the provision status for each of the provided guids one by one. I will then log the status accordingly and finally output the results in both a table and JSON format.

Action: get_provision_status: 1adr4
PAUSE

Step 1
Observation: 1adr4 status: INFO: Initializing
Action: log_status: INFO: Initializing
PAUSE

Step 2
INFO: Initializing Logged status to Slack.
Observation: 0
Action: get_provision_status: aabf5
PAUSE

Step 3
Observation: aabf5 status: INFO: Initializing
Action: log_status: INFO: Initializing
PAUSE

Step 4
INFO: Initializing Logged status to Slack.
Observation: 0
Action: get_provision_status: 45663
PAUSE

Step 5
Observation: 45663 status: INFO: Initializing
Action: log_status: INFO: Initializing
PAUSE

Step 6
INFO: Initializing Logged status to Slack.
Observation: 0
Action: get_provision_status: 45ghb
PAUSE

Step 7
Observation: 45ghb status: ERROR: API Timeout
Action: log_error: ERROR: API Timeout
PAUSE

Step 8
ERROR: API Timeout Log